In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
!unzip amexpert-2019.zip

In [0]:
import pandas as pd
train = pd.read_csv('amexpert-2019/train.csv')
campaign_data = pd.read_csv('amexpert-2019/campaign_data.csv')
cutomer_data = pd.read_csv('amexpert-2019/customer_demographics.csv')
transaction_data = pd.read_csv('amexpert-2019/customer_transaction_data.csv')
item_data = pd.read_csv('amexpert-2019/item_data.csv')
coupon_item = pd.read_csv('amexpert-2019/coupon_item_mapping.csv')
test = pd.read_csv('amexpert-2019/test_QyjYwdj.csv')


In [0]:
train.shape

In [0]:
test.shape

In [0]:
campaign_data['start_date'] =  pd.to_datetime(campaign_data['start_date'],format  = '%d/%m/%y')
campaign_data['end_date'] =  pd.to_datetime(campaign_data['end_date'],format  = '%d/%m/%y')

In [0]:
campaign_data['duration'] = (campaign_data['end_date']-campaign_data['start_date']).dt.days

In [0]:
campaign_data.sort_values(by = ['start_date','end_date'],inplace = True)

In [0]:
campaign_data = pd.merge(campaign_data,(campaign_data['start_date']-campaign_data['start_date'].shift(1)).dt.days,left_index=True,right_index = True )

In [0]:
campaign_data.rename(columns={'start_date_x':'start_date','start_date_y':'days_from_start_of_last_campaign'},inplace = True)

In [0]:
campaign_data['days_from_end_of_last_campaign']=(campaign_data['start_date']-campaign_data['end_date'].shift(1)).dt.days

In [0]:
campaign_data['days_from_end_of_last_campaign']=campaign_data['days_from_end_of_last_campaign'].fillna(0)
campaign_data['days_from_start_of_last_campaign']= campaign_data['days_from_start_of_last_campaign'].fillna(0)

In [0]:
train = train.merge(campaign_data,on = 'campaign_id')
train

In [0]:
test = test.merge(campaign_data,on ='campaign_id')
test

In [0]:
print(train['end_date'].max(),test['start_date'].min())

In [0]:
item_data['type'] = item_data['brand_type']+'_'+item_data['category']

In [0]:
items_per_brand = item_data.groupby('brand')['item_id'].nunique().to_dict()
category_per_brand = item_data.groupby('brand')['category'].nunique().to_dict()

In [0]:
item_data['items_per_brand'] = item_data['brand'].map(items_per_brand)
item_data['category_per_brand'] = item_data['brand'].map(category_per_brand)

In [0]:
coupon_item['item_id'].nunique()

In [0]:
(train['coupon_id']).nunique()

In [0]:
coupon_item = coupon_item.merge(item_data,on = 'item_id')

In [0]:
coupon_item['coupon_id'].unique()

In [0]:
coupon_item['items_per_coupon'] = coupon_item['coupon_id'].map(coupon_item.groupby('coupon_id')['item_id'].nunique().to_dict())
coupon_item['coupons_per_item'] = coupon_item['item_id'].map(coupon_item.groupby('item_id')['coupon_id'].nunique().to_dict())

In [0]:
customer_data = cutomer_data

In [0]:
transaction_data['date'] = pd.to_datetime(transaction_data['date'],format = '%Y-%m-%d')

In [0]:
transaction_data['month'] = (transaction_data['date'].dt.month)
transaction_data['day_of_week'] = (transaction_data['date'].dt.dayofweek)
transaction_data['ismonthend'] = (transaction_data['date'].dt.day > 25).astype('int')
transaction_data['ismonthbeginning'] = (transaction_data['date'].dt.day < 7).astype('int')

In [0]:
transaction_data['total_discount'] = transaction_data['other_discount']+transaction_data['coupon_discount']

In [0]:
transaction_data['cost_price'] = transaction_data['selling_price']-transaction_data['total_discount']
transaction_data['discount_prec'] = abs(transaction_data['total_discount'])/(transaction_data['cost_price'])

In [0]:
transaction_data['before_coupon_price'] = transaction_data['selling_price']- transaction_data['other_discount']
transaction_data['is_coupon_present'] = (abs(transaction_data['coupon_discount'])>0).astype('int')
transaction_data.sort_values(by = 'date',inplace = True)


In [0]:
transaction_data['selling_price_ewm'] = transaction_data.groupby('customer_id')['selling_price'].apply(lambda x: x.ewm(halflife=2).mean())

In [0]:
transaction_data = transaction_data.merge(customer_data,on = 'customer_id')

In [0]:
transaction_data['other_discount_ewm'] = transaction_data.groupby('customer_id')['other_discount'].apply(lambda x: x.ewm(halflife=2).mean())
transaction_data['coupon_discount_ewm'] = transaction_data.groupby('customer_id')['coupon_discount'].apply(lambda x: x.ewm(halflife=2).mean())
transaction_data['total_discount_ewm'] = transaction_data.groupby('customer_id')['total_discount'].apply(lambda x: x.ewm(halflife=2).mean())
transaction_data['before_coupon_price_ewm'] = transaction_data.groupby('customer_id')['before_coupon_price'].apply(lambda x: x.ewm(halflife=2).mean())
transaction_data['cost_price_ewm'] = transaction_data.groupby('customer_id')['cost_price'].apply(lambda x: x.ewm(halflife=2).mean())
transaction_data['discount_prec_ewm'] = transaction_data.groupby('customer_id')['discount_prec'].apply(lambda x: x.ewm(halflife=2).mean())

In [0]:
transaction_data = transaction_data.merge(item_data,on = 'item_id')

In [0]:
transaction_data.columns

In [0]:
df=pd.concat((train,test),axis=0)

In [0]:
temp=df[['customer_id','start_date','coupon_id','id']]

In [0]:
def return_stats(x):
    return [x.mean(),x.sum(),x.std(),x.max()]

In [0]:
df['customer_id'].nunique()

In [0]:
df.shape

In [0]:
train.columns

In [0]:
col = ['quantity', 'selling_price',
       'other_discount', 'coupon_discount','total_discount', 'cost_price',
       'discount_prec', 'before_coupon_price', 'is_coupon_present',
       'selling_price_ewm','other_discount_ewm',
       'coupon_discount_ewm', 'total_discount_ewm', 'before_coupon_price_ewm',
       'cost_price_ewm', 'discount_prec_ewm','items_per_brand', 'category_per_brand']

In [0]:
for c in col :
    temp[c+'_mean'] = 0
    temp[c+'_max'] = 0
    temp[c+'_std'] = 0
    temp[c+'_sum'] = 0

In [0]:
from tqdm.auto import tqdm

In [0]:
for customer in tqdm(temp['customer_id'].unique()) :
    t1 = temp.loc[temp['customer_id'] == customer]
    for date in (t1['start_date'].unique()):
        t2 = transaction_data.loc[(transaction_data['customer_id'] == customer)&(transaction_data['date']<date)]
        if not t2.empty:
            for c in col :
                temp.loc[(temp['customer_id']==customer)& (temp['start_date']==date),c+'_mean'] = t2[c].mean()
                temp.loc[(temp['customer_id']==customer)& (temp['start_date']==date),c+'_max'] = t2[c].max()
                temp.loc[(temp['customer_id']==customer)& (temp['start_date']==date),c+'_std'] = t2[c].std()
                temp.loc[(temp['customer_id']==customer)& (temp['start_date']==date),c+'_sum'] = t2[c].sum()




In [0]:
temp['id'].nunique()

In [0]:
temp.to_csv('store')

In [0]:
train.shape

In [0]:
temp.columns

In [0]:
temp2 = temp.drop(columns = ['customer_id', 'start_date', 'coupon_id'])

In [0]:
train= train.merge(temp2,on ='id',how = 'left')

In [0]:
test = test.merge(temp2,on = 'id',how = 'left')

In [0]:
test.shape

In [0]:
train.shape

In [0]:
train = train.merge(customer_data,on='customer_id',how = 'left')


In [0]:
train.shape

In [0]:
test = test.merge(customer_data,on='customer_id',how = 'left')

In [0]:
test.shape

In [0]:
coupon_item.head()

In [0]:
coupon_item.shape

In [0]:
mapi = pd.read_csv('amexpert-2019/coupon_item_mapping.csv')

In [0]:
p = train.merge(mapi,on='coupon_id',how = 'right')

In [0]:
p.shape

In [0]:
train.shape

In [0]:
test.shape

In [0]:
train.columns

In [0]:
train.to_csv('Final_Training_set')
test.to_csv('Final_test_set')